In [9]:
import pygame
import time

pygame.init()

window_width = 800
window_height = 600
fon = 'fon.png'

window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Игра v1.0")

speed = 0
shift_background = 0

try:
    img1 = pygame.image.load(fon)
    back_fon = pygame.transform.scale(img1, (window_width, window_height))
except Exception as e:
    print(f"Ошибка: {e}")
    pygame.quit()
    exit()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                speed = -5    # Движение фона влево
            elif event.key == pygame.K_RIGHT:
                speed = 5     # Движение фона вправо 
                
        elif event.type == pygame.KEYUP:
            if event.key in (pygame.K_LEFT, pygame.K_RIGHT):
                speed = 0     

    shift_background = (shift_background + speed) % window_width
    
    window.blit(back_fon, (shift_background, 0))
    if shift_background != 0:
        window.blit(back_fon, (shift_background - window_width, 0))
    
    pygame.display.update()
    time.sleep(0.02)

pygame.quit()

In [1]:
import pygame as pg

pg.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
BACKGROUND_SPEED = 1


class Player(pg.sprite.Sprite):
    def __init__(self, filename, hero_x=100, hero_y=250, x_speed=0, y_speed=0):
        pg.sprite.Sprite.__init__(self)
        self.image = pg.image.load(filename)
        self.rect = self.image.get_rect()
        self.hero_x = hero_x
        self.hero_y = hero_y
        self.rect.x = hero_x 
        self.rect.y = hero_y
        self.x_speed = x_speed
        self.y_speed = y_speed

    def update(self):
        self.rect.x += self.x_speed

        # проверка, чтобы персонаж не выходил за границы экрана по X
        if self.rect.left < 0:
            self.rect.left = 0
        elif self.rect.right > WINDOW_WIDTH:
            self.rect.right = WINDOW_WIDTH

        self.rect.y += self.y_speed

        # проверка, чтобы персонаж не выходил за границы экрана по Y
        if self.rect.top < 0:
            self.rect.top = 0
        elif self.rect.bottom > WINDOW_HEIGHT:
            self.rect.bottom = WINDOW_HEIGHT

            
window = pg.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pg.display.set_caption("Аркадная игра")

hero = Player('artillery(1).png')

background_img = pg.image.load('fon.png')
background_img = pg.transform.scale(background_img, (WINDOW_WIDTH, WINDOW_HEIGHT))

background_x = 0
background_speed = 0
player_hero_speed = 1 

running = True
while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        elif event.type == pg.KEYDOWN:
            if event.key == pg.K_UP:
                hero.y_speed = -player_hero_speed
            elif event.key == pg.K_DOWN:
                hero.y_speed = player_hero_speed
            elif event.key == pg.K_LEFT:
                hero.x_speed = -player_hero_speed
                background_speed = hero.x_speed*0.5  
            elif event.key == pg.K_RIGHT:
                hero.x_speed = player_hero_speed
                background_speed = hero.x_speed*0.5
        elif event.type == pg.KEYUP:
            if event.key in (pg.K_LEFT, pg.K_RIGHT):
                hero.x_speed = 0
                background_speed = 0
            elif event.key in (pg.K_UP, pg.K_DOWN):
                hero.y_speed = 0

    hero.update()

    if (hero.rect.left <= 0) or (hero.rect.right >= WINDOW_WIDTH):
        try:
            background_x -= BACKGROUND_SPEED * hero.x_speed / abs(hero.x_speed)
        except:
            pass
        if background_x >= WINDOW_WIDTH:
            background_x -= WINDOW_WIDTH
        elif background_x <= -WINDOW_WIDTH:
            background_x += WINDOW_WIDTH

    window.blit(background_img, (background_x, 0))
    if background_x > 0:
        window.blit(background_img, (background_x - WINDOW_WIDTH, 0))
    else:
        window.blit(background_img, (background_x + WINDOW_WIDTH, 0))

    window.blit(hero.image, hero.rect)

    pg.display.flip()

pg.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
import pygame as pg

pg.init()

WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
BACKGROUND_SPEED = 1
ARROW_SPEED = 7

# игрок
class Player(pg.sprite.Sprite):
    def __init__(self, filename, x=100, y=250):
        super().__init__()
        self.image = pg.image.load(filename)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.x_speed = 0
        self.y_speed = 0

    def update(self):
        self.rect.x += self.x_speed
        self.rect.y += self.y_speed
        
        self.rect.left = max(0, self.rect.left)
        self.rect.right = min(WINDOW_WIDTH, self.rect.right)
        self.rect.top = max(0, self.rect.top)
        self.rect.bottom = min(WINDOW_HEIGHT, self.rect.bottom)

# враг
class Enemy(pg.sprite.Sprite):
    def __init__(self, filename, x, y):
        super().__init__()
        self.image = pg.image.load(filename)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.speed = 2

    def update(self):
        self.rect.x += self.speed
        if self.rect.right > WINDOW_WIDTH or self.rect.left < 0:
            self.speed *= -1

# стрела
class Arrow(pg.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pg.Surface((20, 5))
        self.image.fill((255, 0, 0))
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.speed = ARROW_SPEED

    def update(self):
        self.rect.x += self.speed
        if self.rect.left > WINDOW_WIDTH:
            self.kill()

window = pg.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pg.display.set_caption("BM-21 'Grad' VS Forces of NATO")

hero = Player('artillery(1).png')
enemy1 = Enemy('enemy.png', 500, 200)
enemy2 = Enemy('enemy.png', 300, 400)
background_img = pg.transform.scale(pg.image.load('fon.png'), (WINDOW_WIDTH, WINDOW_HEIGHT))

all_sprites = pg.sprite.Group()
enemies = pg.sprite.Group()
arrows = pg.sprite.Group()

all_sprites.add(hero, enemy1, enemy2)
enemies.add(enemy1, enemy2)

background_x = 0
running = True
clock = pg.time.Clock()

while running:
    dt = clock.tick(60)
    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            
        if event.type == pg.KEYDOWN:
            if event.key == pg.K_UP:
                hero.y_speed = -3
            elif event.key == pg.K_DOWN:
                hero.y_speed = 3
            elif event.key == pg.K_LEFT:
                hero.x_speed = -3
            elif event.key == pg.K_RIGHT:
                hero.x_speed = 3
            elif event.key == pg.K_SPACE:
                new_arrow = Arrow(hero.rect.right, hero.rect.centery)
                all_sprites.add(new_arrow)
                arrows.add(new_arrow)

        if event.type == pg.KEYUP:
            if event.key in (pg.K_LEFT, pg.K_RIGHT):
                hero.x_speed = 0
            elif event.key in (pg.K_UP, pg.K_DOWN):
                hero.y_speed = 0

    all_sprites.update()

    hits = pg.sprite.groupcollide(enemies, arrows, True, True)
    
    window.blit(background_img, (background_x, 0))
    all_sprites.draw(window)
    pg.display.flip()

pg.quit()